In [1]:
!pip install mkl faiss-cpu langchain langchain-community langchain-chroma google-generativeai InstructorEmbedding


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.7/169.7 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 15.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.8/184.8 MB 15.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 46.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 53.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 35.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import numpy as np
import faiss

In [5]:
from langchain_community.document_loaders import HuggingFaceDatasetLoader

trec_covid_loader = HuggingFaceDatasetLoader("BeIR/trec-covid", "text", "corpus")
trec_covid_data =  trec_covid_loader.load()

/usr/local/lib/python3.11/dist-packages/datasets/load.py:2089: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=None' instead.
  warnings.warn(


In [6]:
trec_covid_data[10]

Document(metadata={'_id': '5tkvsudh', 'title': 'Conservation of polyamine regulation by translational frameshifting from yeast to mammals'}, page_content='"Regulation of ornithine decarboxylase in vertebrates involves a negative feedback mechanism requiring the protein antizyme. Here we show that a similar mechanism exists in the fission yeast Schizosaccharomyces pombe. The expression of mammalian antizyme genes requires a specific +1 translational frameshift. The efficiency of the frameshift event reflects cellular polyamine levels creating the autoregulatory feedback loop. As shown here, the yeast antizyme gene and several newly identified antizyme genes from different nematodes also require a ribosomal frameshift event for their expression. Twelve nucleotides around the frameshift site are identical between S.pombe and the mammalian counterparts. The core element for this frameshifting is likely to have been present in the last common ancestor of yeast, nematodes and mammals."')

In [29]:
import pickle
with open("idx_to_pubmedID.pkl", "rb") as file:
    idx_to_pubmedID = pickle.load(file)

In [52]:
##### from InstructorEmbedding import INSTRUCTOR
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_chroma import Chroma
import chromadb.utils.embedding_functions as embedding_functions
from dotenv import load_dotenv
from langchain.docstore.document import Document
import os
import google.generativeai as genai
from bs4 import BeautifulSoup
import requests
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from typing import Any, List, Optional

from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import GenerationChunk
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain.load import dumps, loads

from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
import torch
import faiss
import numpy as np
from datasets import load_dataset
from concurrent.futures import ThreadPoolExecutor


ds = load_dataset("HPAI-BSC/medqa-cot")



index = faiss.IndexFlatIP(768)
#trec_covid_embed
trec_covid_embeds = np.load("trec_covid_embed.npy")
# faiss.normalize_L2(trec_covid_embeds) # buat cosine
index.add(trec_covid_embeds[:, 0, :])




query_model = AutoModel.from_pretrained("ncbi/MedCPT-Query-Encoder")
query_tokenizer = AutoTokenizer.from_pretrained("ncbi/MedCPT-Query-Encoder")

load_dotenv()


from InstructorEmbedding import INSTRUCTOR
instructor_model = INSTRUCTOR('hkunlp/instructor-xl')

os.environ["GEMINI_API_KEY"] = ""


model_name = "hkunlp/instructor-xl"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
# https://arxiv.org/abs/2212.09741
instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="Represent the Medicine sentence for retrieving relevant documents:",
)

instructor_embeddings.query_instruction = (
    "Represent the Medicine sentence for retrieving relevant documents: "
)

#"Represent the Medicine sentence for retrieving relevant documents: ",

tokenizer = AutoTokenizer.from_pretrained("ncbi/MedCPT-Cross-Encoder")
model = AutoModelForSequenceClassification.from_pretrained("ncbi/MedCPT-Cross-Encoder")





medqa_cot_data = ds["train"]



def rerank_docs_medcpt(question, docs):
  
    relevant = set()

    def process_query(query):
        pairs = [[query, article] for article in docs]
        with torch.no_grad():
            encoded = tokenizer(
                pairs,
                truncation=True,
                padding=True,
                return_tensors="pt",
                max_length=512,
            )

            logits = model(**encoded).logits.squeeze(dim=1)
            values, indices = torch.sort(logits, descending=True)
            curr_relevant = [docs[i] for i in indices[:10]]
        return curr_relevant, values

    
    results, values = process_query(question)

    for curr_relevant in results:
        relevant.add(curr_relevant)

    relevant = list(relevant)
    return relevant, values




def retrieve_and_append(query):

    instruction = "Represent the Medicine sentence for retrieving relevant documents:"
    q_embeddings = instructor_model.encode([[instruction,query]])
    # faiss.normalize_L2(q_embeddings) # buat cosine

    
    scores, inds = index.search(q_embeddings, k=15)
    relevant_docs = []
    for idx in inds[0]:
        curr = trec_covid_data[idx]
        relevant_docs.append(curr.metadata["title"] + curr.page_content)
    return scores, inds, relevant_docs

def retrieve_medqa_cot(query):
    cots = medqa_cot_retriever.invoke(query)
    return cots

def answer(question):
    docs = {}
    docs["chroma"] = []
    search_query = [question]

    inds = []
    with ThreadPoolExecutor(max_workers=1) as executor:
        results = executor.map(retrieve_and_append, search_query)
    for new_contexts in results:
        docs["chroma"].extend(new_contexts[2])
        inds.extend(new_contexts[1])
        
    

  
    chroma_docs = [doc for doc in docs["chroma"]]

    docs = list(set(chroma_docs))
    inds=  inds[0]
    new_inds = []
    for i in range(len(inds)):
        new_inds.append(idx_to_pubmedID[inds[i]])
    str_to_id = dict(map(lambda x: (x[0], x[1]), zip(docs, new_inds)))
    
    relevant_docs, values = rerank_docs_medcpt(question, docs)
    
    relevant_ids = []
    for doc in relevant_docs:
        relevant_ids.append(str_to_id[doc])
    
    return relevant_docs, values, relevant_ids





load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
# !unzip trec-covid.zip

In [30]:
# query trec-covid pakai text

In [12]:
import pandas as pd

df = pd.read_csv('./trec-covid/qrels/test.tsv', sep='\t')

df.head()

,query-id,corpus-id,score
0,1,005b2j4b,2
1,1,00fmeepz,1
2,1,g7dhmyyo,2
3,1,0194oljo,1
4,1,021q9884,1


In [71]:
qrels = {}


In [72]:
for key, item in df.iterrows():
    query_id = item["query-id"]
    corpus_id = item["corpus-id"]
    score = item["score"]
    if f"{query_id}" not in qrels:
        qrels[f"{query_id}"] = {}
    qrels[f"{query_id}"][corpus_id] = score
    

In [15]:
queries = {}

In [16]:
import json
with open("./trec-covid/queries.jsonl", encoding='utf8') as fIn:
    for line in fIn:
        line = json.loads(line)
        queries[line.get("_id")] = line.get("text")

In [53]:
results = {}

In [54]:
for key, query in queries.items():
    
    relevant_docs, scores, relevant_ids = answer(query)
    if len(results) % 10 == 0:
        print(f"query: {query}, relevants: {relevant_ids}")
    results[key] = {}
    for i in range(len(relevant_ids)):
        rel_ids = relevant_ids[i]
        results[key][rel_ids] = scores[i].item()
    

query: what is the origin of COVID-19, relevants: ['jb05x03a', 'gdfxiosc', 'x687hgqs', '6bzoyr46', '2psttxu3', 'b1r2bydo', '1q2gqh22', 'dl8gahmn', '8lmrft9m', '43idmfnl']
query: what are the guidelines for triaging patients infected with coronavirus?, relevants: ['8c1s89dc', 'jv3xx09f', '4jag7lzb', '9tlk02y6', 'i2t2188i', '4f022e55', 'xizlc7rf', 'k7ngv0n7', '5c6lwhno', '8t7pos8x']
query: what are the mortality rates overall and in specific populations, relevants: ['5ep75qzn', 'mc2r500i', '3jmo35jc', 't9qdiz7n', '7atoko6c', 'ekxz577j', 'r086n6hq', '10e0srnx', '852w35od', 'f3eogz0n']
query: How does the coronavirus differ from seasonal flu?, relevants: ['286ca29i', '6wsiyvag', 'x56ybeas', 'j1cdoxqs', 'chb4iuis', 'gc3iq0wp', '81m48udk', '0new927o', 'teu23mzu', 'ik9rr2ed']
query: What are the impacts of COVID-19 among African-Americans that differ from the rest of the U.S. population?, relevants: ['tzg9qnt4', 'ey1sqch4', 'dwriseo6', 'u7s67aug', 'nahhmgqi', 'bgpep5lc', 'ujlsdpzf', 'i2b9py8e

In [76]:
k_values=  [1, 3, 5, 10]

In [77]:
ndcg = {}
_map = {}
recall = {}
precision = {}

for k in k_values:
    ndcg[f"NDCG@{k}"] = 0.0
    _map[f"MAP@{k}"] = 0.0
    recall[f"Recall@{k}"] = 0.0
    precision[f"P@{k}"] = 0.0

map_string = "map_cut." + ",".join([str(k) for k in k_values])
ndcg_string = "ndcg_cut." + ",".join([str(k) for k in k_values])
recall_string = "recall." + ",".join([str(k) for k in k_values])
precision_string = "P." + ",".join([str(k) for k in k_values])

In [80]:
import scipy

In [83]:

!pip install --upgrade pip setuptools wheel 

!pip install pytrec_eval


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached setuptools-75.6.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.8 MB/s eta 0:00:00 0:00:01
Using cached setuptools-75.6.0-py3-none-any.whl (1.2 MB)
Using cached wheel-0.45.1-py3-none-any.whl (72 kB)
  Attempting uninstall: wheel
    Found existing installation: wheel 0.35.1
    Uninstalling wheel-0.35.1:
      Successfully uninstalled wheel-0.35.1
  Attempting uninstall: setuptools
    Found existing installation: setuptools 69.0.3
    Uninstalling setuptools-69.0.3:
      Successfully uninstalled setuptools-69.0.3
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient-utils 0.5.0 requires wheel<

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached pytrec_eval-0.5.tar.gz (15 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp311-cp311-linux_x86_64.whl size=308656 sha256=8a51ccbffdd79e49c1064186c42772a4b7e59f64ef89b6298ff468b4ca484fbe
  Stored in directory: /root/.cache/pip/wheels/0f/89/42/86aecdb99975f1840c27bc37fdfed72116abcf82e2c9dc76a8
Successfully built pytrec_eval


In [84]:
import pytrec_eval

evaluator = pytrec_eval.RelevanceEvaluator(
            qrels, {map_string, ndcg_string, recall_string, precision_string}
        )
scores = evaluator.evaluate(results)


In [86]:
df_score = pd.DataFrame(scores).T
df_score.index.name = 'queryID'

df_score

,P_1,P_3,P_5,P_10,recall_1,recall_3,recall_5,recall_10,ndcg_cut_1,ndcg_cut_3,ndcg_cut_5,ndcg_cut_10,map_cut_1,map_cut_3,map_cut_5,map_cut_10
queryID,,,,,,,,,,,,,,,,
1,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1.0,0.666667,0.6,0.4,0.003185,0.006369,0.009554,0.012739,1.0,0.703918,0.639945,0.488645,0.003185,0.005308,0.007219,0.009039
3,0.0,0.666667,0.6,0.7,0.000000,0.003185,0.004777,0.011146,0.0,0.530721,0.456601,0.505613,0.000000,0.001858,0.003052,0.007427
4,0.0,0.000000,0.0,0.2,0.000000,0.000000,0.000000,0.003781,0.0,0.000000,0.000000,0.139618,0.000000,0.000000,0.000000,0.000690
5,0.0,0.333333,0.2,0.1,0.000000,0.001629,0.001629,0.001629,0.0,0.234639,0.169580,0.110046,0.000000,0.000543,0.000543,0.000543
6,0.0,0.000000,0.0,0.3,0.000000,0.000000,0.000000,0.003128,0.0,0.000000,0.000000,0.199306,0.000000,0.000000,0.000000,0.000675
7,0.0,0.333333,0.6,0.5,0.000000,0.002079,0.006237,0.010395,0.0,0.234639,0.446854,0.431996,0.000000,0.000693,0.002980,0.005405
8,0.0,0.333333,0.4,0.5,0.000000,0.001582,0.003165,0.007911,0.0,0.296082,0.360055,0.410385,0.000000,0.000791,0.001582,0.003956
9,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [100]:
df_score.describe()

,P_1,P_3,P_5,P_10,recall_1,recall_3,recall_5,recall_10,ndcg_cut_1,ndcg_cut_3,ndcg_cut_5,ndcg_cut_10,map_cut_1,map_cut_3,map_cut_5,map_cut_10
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.320000,0.360000,0.352000,0.352000,0.000913,0.003122,0.004707,0.009017,0.280000,0.312458,0.311330,0.313691,0.000913,0.002369,0.003208,0.005306
std,0.471212,0.292305,0.230563,0.185428,0.001759,0.004481,0.005817,0.008839,0.430472,0.250398,0.204913,0.172061,0.001759,0.004138,0.005169,0.007373
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.200000,0.200000,0.000000,0.000000,0.001671,0.003816,0.000000,0.000000,0.151946,0.200759,0.000000,0.000000,0.000660,0.001457
50%,0.000000,0.333333,0.400000,0.400000,0.000000,0.001823,0.003594,0.007173,0.000000,0.296082,0.339160,0.333543,0.000000,0.001387,0.002371,0.004107
75%,1.000000,0.666667,0.400000,0.500000,0.001359,0.004118,0.005472,0.010707,0.500000,0.469279,0.466924,0.431723,0.001359,0.002575,0.003502,0.005753
max,1.000000,1.000000,0.800000,0.700000,0.009009,0.027027,0.036036,0.054054,1.000000,0.765361,0.786014,0.718342,0.009009,0.027027,0.034234,0.049464


In [88]:
!pip install dataframe-image

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 136.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 156.0 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 0.57.0
    Uninstalling websocket-client-0.57.0:
      Successfully uninstalled websocket-client-0.57.0
  Attempting uninstall: yarl
    Found existing installation: yarl 1.9.4
    Uninstalling yarl-1.9.4:
      Successfully uninstalled yarl-1.9.4
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.9.1
    Uninstalling aiohttp-3.9.1:
      Successfully uninstalled aiohttp-3.9.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient 2.0.6 requires attrs<=19, but you have attrs 23.1.0 which is incompatible.
gradient 2.0.6 requires marshmallow<3.0, but you have marshmallow 3.23.1 which is incompat

In [97]:
# !wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb

# !sudo apt install ./google-chrome-stable_current_amd64.deb -y



In [98]:
import dataframe_image as dfi
dfi.export(df_score, 'df_score.png')
